# Constructing Dissimilarity Biased Corrected (Dbc) and Density Corrected (Ddc)

The classical Dissimilarity Index (D) can be severely upward biased since it is a sum of absolute values and, thus, can lead to greates values of D when small unit are presented even under random even allocation, that is, $p_j^1 = p_j^0$ for all $j$ and $D_{pop} = 0$. For more information, see *Allen, Rebecca, et al. "More reliable inference for the dissimilarity index of segregation." The econometrics journal 18.1 (2015): 40-66.*

## Dissimilarity Bias-Corrected

In [253]:
import pandas as pd
import numpy as np
from scipy.stats import norm

import os

os.chdir('C:/Users/renan/Desktop/inequality-segregation-supplementary-files/')

def calculate_dissim(data, group_pop_var, total_pop_var, b = 0.5, m = 1000):
    data = data.rename(columns={group_pop_var: 'group_pop_var', total_pop_var: 'total_pop_var'})
    T = data.total_pop_var.sum()
    P = data.group_pop_var.sum() / T
    data = data.assign(xi = data.group_pop_var,
                       yi = data.total_pop_var - data.group_pop_var,
                       ti = data.total_pop_var,
                       pi = np.where(data.total_pop_var == 0, 0, data.group_pop_var/data.total_pop_var))
    D = (((data.total_pop_var * abs(data.pi - P)))/ (2 * T * P * (1 - P))).sum()
    return D

In [254]:
census_2010 = pd.read_csv('data/std_2010_fullcount.csv', encoding = "ISO-8859-1", sep = ",")
df = census_2010.loc[census_2010.county == "Riverside County"][['trtid10','tract','pop10','nhblk10']]
df['other_group_pop'] = df.pop10 - df.nhblk10
df.head()

,trtid10,tract,pop10,nhblk10,other_group_pop
5727,6065042012,Census Tract 420.12,6242,677,5565
5729,6065041911,Census Tract 419.11,10258,844,9414
5731,6065041910,Census Tract 419.10,6342,405,5937
5733,6065040816,Census Tract 408.16,2594,346,2248
5735,6065040815,Census Tract 408.15,3586,429,3157


In [255]:
D = calculate_dissim(df, 'nhblk10', 'pop10')
D

0.31565682496226544

In [256]:
B = 500

In [257]:
np.random.seed(1234)

# Group 0: minority group

p0_i = (df.nhblk10 / df.nhblk10.sum())
n0 = df.nhblk10.sum()
sim0 = np.random.multinomial(n0, p0_i, size = B)

# Group 1: complement group
p1_i = (df.other_group_pop / df.other_group_pop.sum())
n1 = df.other_group_pop.sum()
sim1 = np.random.multinomial(n1, p1_i, size = B)

In [258]:
Dbcs = np.empty(B)
for i in np.array(range(B)):
    data_aux = {'simul_group': sim0[i].tolist(), 'simul_tot': (sim0[i] + sim1[i]).tolist()}
    df_aux = pd.DataFrame.from_dict(data_aux)
    Dbcs[i] = calculate_dissim(df_aux, 'simul_group', 'simul_tot')
Db = Dbcs.mean()
Db

0.31646728910576133

In [259]:
Dbc = 2 * D - Db
Dbc # It expected to be lower than D, because D is upwarded biased

0.31484636081876954

## Dissimilarity Density-Corrected

The bias correction works well if the bias is constant for different values of $D_{pop}$. The density correction works with the asymptotic distribution $D$ which is not trivial due to the bias and, therefore, the distribution of $\sqrt{n^c}(\hat{p}_j^c - p_j^c)$ is not normal with zero expectation.

The distribution of $D$ is related to the folded normal distribution because of the sum of absolute values.

In [205]:
sigma_hat_j = np.sqrt(((p1_i * (1 - p1_i)) / n1) + ((p0_i * (1 - p0_i)) / n0))
theta_hat_j = abs(p1_i - p0_i) / sigma_hat_j

In [206]:
# INVERSE OF THE FUNCTION THAT NEEDS TO BE MINIMIZED
theta_j = 1.2
def fold_norm(x):
    y = (-1) * norm.pdf(x - theta_j) + norm.pdf(x + theta_j)
    return y

In [207]:
initial_guesses = np.array(0)
res = minimize(fold_norm, 
               initial_guesses, 
               method='nelder-mead',
               options = {'xtol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: -0.379442
         Iterations: 39
         Function evaluations: 80


In [216]:
res.final_simplex[0][1][0] # Point where minimum occurs
#res.final_simplex[1][1] # Minimum value of the function

1.3085258712768568

Constructing function that returns $n(\hat{\theta}_j)$

In [217]:
def return_optimal_theta(theta_j):
    
    def fold_norm(x):
        y = (-1) * norm.pdf(x - theta_j) + norm.pdf(x + theta_j)
        return y
    
    initial_guesses = np.array(0)
    res = minimize(fold_norm, 
                   initial_guesses, 
                   method='nelder-mead',
                   options = {'xtol': 1e-8})
    return res.final_simplex[0][1][0]

In [218]:
return_optimal_theta(1.5)

1.5307052841186537

In [226]:
# Applying a function over all elements of a pandas Series
optimal_thetas = theta_hat_j.apply(return_optimal_theta)
optimal_thetas

5727     1.026590e+01
5729     5.343340e+00
5731     1.364639e+00
5733     9.657815e+00
5735     9.457704e+00
5737     1.524833e+01
5739     1.378897e+01
5741     2.511840e+01
5743     1.428966e+00
5745     5.459233e+00
5747     2.311372e+01
5749     1.969688e+01
5751     2.735606e+01
5753     1.192382e+01
5755     1.669856e+00
5757     3.679724e+00
5759     7.637636e+00
5761     1.181816e+01
5763     7.420035e+00
5765     1.343866e+01
5767     8.367763e+00
5769     1.112624e+01
5771     1.071177e+00
5773     1.036354e+01
5775     3.664352e+01
5777     1.725656e+01
5779     8.746915e+00
5781     2.617593e+01
5783     3.437409e+01
5785     1.676957e+01
             ...     
12757    1.349969e+01
12762    4.817937e+00
12765    8.356934e+00
12767    3.283314e+01
12771    2.158831e+01
12773    1.173930e+01
12775    4.255239e+00
12777    2.923079e+01
12779    2.010700e+01
12784    3.439865e+01
12787    2.194113e+01
12792    1.747169e+01
12794    2.600516e+01
12796    7.629395e-09
12801    3

In [231]:
Ddc = (sigma_hat_j * optimal_thetas).sum() / 2
Ddc

0.2954881746588274